In [20]:
import random

# Read the full labeled dataset
with open("output_3.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Shuffle to randomize
random.shuffle(lines)

# Split: 80% for train, 20% for test
split_idx = int(0.8 * len(lines))
train_data = lines[:split_idx]
test_gold = lines[split_idx:]

# Save train data (for your HMM training)
with open("train_data.txt", "w", encoding="utf-8") as f:
    f.writelines(train_data)

# Save gold test data (correct tags for accuracy checking)
with open("test_gold.txt", "w", encoding="utf-8") as f:
    f.writelines(test_gold)

# Create test_input.txt: only words, tags removed
with open("test_input.txt", "w", encoding="utf-8") as f:
    for line in test_gold:
        tokens = line.strip().split()
        words = [token.rsplit("/", 1)[0] for token in tokens if "/" in token]
        f.write(" ".join(words) + "\n")


In [27]:
import sys
import math
from decimal import *
import codecs

# --- NEW: Define a constant for Lidstone Smoothing ---
# Using a small value like 0.1 is better than add-one smoothing.
LAMBDA = Decimal(0.1)

tag_list = set()
tag_count = {}
word_set = set()


# --- NEW: Function to build a lexicon of known words and their possible tags ---
def build_lexicon(train_data):
    lexicon = {}
    for sentence in train_data:
        for token in sentence:
            # Safely split the token
            parts = token.rsplit('/', 1)
            if len(parts) == 2:
                word, tag = parts
                word = word.lower()
                if word not in lexicon:
                    lexicon[word] = set()
                lexicon[word].add(tag)
    return lexicon


# --- NEW: Function to build a model for handling unknown words via suffixes ---
def build_suffix_model(train_data, suffix_len=3):
    suffix_tag_counts = {}
    suffix_totals = {}

    # Count suffix-tag co-occurrences
    for sentence in train_data:
        for token in sentence:
            parts = token.rsplit('/', 1)
            if len(parts) == 2:
                word, tag = parts
                word = word.lower()
                if len(word) > suffix_len:
                    suffix = word[-suffix_len:]
                    # Count (suffix, tag) pair
                    if suffix not in suffix_tag_counts:
                        suffix_tag_counts[suffix] = {}
                    suffix_tag_counts[suffix][tag] = suffix_tag_counts[suffix].get(tag, 0) + 1
                    # Count total occurrences of the suffix
                    suffix_totals[suffix] = suffix_totals.get(suffix, 0) + 1

    # Convert counts to probabilities
    suffix_probabilities = {}
    for suffix, tag_counts in suffix_tag_counts.items():
        suffix_probabilities[suffix] = {}
        total = suffix_totals[suffix]
        for tag, count in tag_counts.items():
            suffix_probabilities[suffix][tag] = Decimal(count) / Decimal(total)

    return suffix_probabilities


def clean_traindata(raw_lines):
    cleaned_data = []
    for line in raw_lines:
        line = line.strip()
        tokens = line.split()
        valid_tokens = []
        for token in tokens:
            if '/' in token and token.count('/') == 1:
                word, tag = token.split('/')
                if word.strip() and tag.strip():
                    valid_tokens.append(token)
        if valid_tokens:
            cleaned_data.append(valid_tokens)
    return cleaned_data


def parse_traindata():
    fin = "output_3.txt"
    output_file = "hmmmodel.txt"

    try:
        with codecs.open(fin, mode='r', encoding="utf-8") as input_file:
            lines = input_file.readlines()
        wordtag_list = clean_traindata(lines)
        return wordtag_list
    except IOError:
        with codecs.open(output_file, mode='w', encoding="utf-8") as fo:
            fo.write("File not found: {}".format(fin))
        sys.exit()


def transition_count(train_data):
    global tag_list, word_set, tag_count
    transition_dict = {}
    tag_count.clear()
    word_set.clear()
    tag_list.clear()


    for value in train_data:
        previous = "start"
        for data in value:
            i = data[::-1]
            word, tag = data.rsplit('/', 1)
            word_set.add(word.lower())
            tag_list.add(tag)
            tag_count[tag] = tag_count.get(tag, 0) + 1
            key = previous + "~tag~" + tag
            transition_dict[key] = transition_dict.get(key, 0) + 1
            previous = tag
    return transition_dict


def transition_probability(train_data):
    count_dict = transition_count(train_data)
    prob_dict = {}
    start_transitions = 0
    tag_transitions = {}

    for key, count in count_dict.items():
        from_tag = key.split("~tag~")[0]
        if from_tag == "start":
            start_transitions += count
        else:
            tag_transitions[from_tag] = tag_transitions.get(from_tag, 0) + count

    for key, count in count_dict.items():
        from_tag = key.split("~tag~")[0]
        if from_tag == "start":
            prob_dict[key] = Decimal(count) / Decimal(start_transitions)
        else:
            prob_dict[key] = Decimal(count) / Decimal(tag_transitions[from_tag])
    return prob_dict


# --- MODIFIED: Use Lidstone smoothing (add-lambda) ---
def transition_smoothing(train_data):
    transition_prob = transition_probability(train_data)
    total_tags = len(tag_list)
    for tag1 in ["start"] + list(tag_list):
        denominator = tag_count.get(tag1, 0) + (LAMBDA * total_tags)
        for tag2 in tag_list:
            key = tag1 + "~tag~" + tag2
            if key not in transition_prob:
                # Apply Lidstone smoothing
                transition_prob[key] = LAMBDA / denominator
    return transition_prob


def emission_count(train_data):
    count_word = {}
    for value in train_data:
        for data in value:
            i = data[::-1]
            word = data[:-i.find("/") - 1].lower()
            tag = data.split("/")[-1]
            key = word + "/" + tag
            count_word[key] = count_word.get(key, 0) + 1
    return count_word


def emission_probability(train_data):
    global tag_count
    word_count = emission_count(train_data)
    emission_prob_dict = {}
    for key in word_count:
        tag = key.split("/")[-1]
        # Smoothing is handled in Viterbi for unknown words,
        # here we use raw probability for known words.
        emission_prob_dict[key] = Decimal(word_count[key]) / Decimal(tag_count[tag])
    return emission_prob_dict


# --- MODIFIED: Main training execution block ---
# Train and build all models
train_data = parse_traindata()
transition_model = transition_smoothing(train_data)
emission_model = emission_probability(train_data)

# --- NEW: Build the lexicon and suffix model for use in the next cell ---
lexicon = build_lexicon(train_data)
suffix_model = build_suffix_model(train_data)


# Save the HMM models (transition and emission) to a file
# Note: The new models (lexicon, suffix_model) are kept in memory for the next step.
fout = codecs.open("hmmmodel.txt", mode='w', encoding="utf-8")
fout.write('Transition Model\n')
for key, value in transition_model.items():
    fout.write('%s:%s\n' % (key, value))

fout.write('Emission Model\n')
for key, value in emission_model.items():
    fout.write('%s:%s\n' % (key, value))

# --- NEW: Also save tag counts for smoothing in Viterbi ---
fout.write('Tag Counts\n')
for key, value in tag_count.items():
    fout.write('%s:%s\n' % (key, value))

fout.close()

print("Training complete. hmmmodel.txt generated.")
print(f"Lexicon contains {len(lexicon)} unique words.")
print(f"Suffix model contains {len(suffix_model)} unique suffixes.")

Training complete. hmmmodel.txt generated.
Lexicon contains 63420 unique words.
Suffix model contains 8943 unique suffixes.


In [28]:
import sys
from decimal import Decimal
import codecs

# --- NEW: Define LAMBDA here as well for consistency ---
LAMBDA = Decimal(0.1)

def parse_models():
    fin = "hmmmodel.txt"
    transition_prob = {}
    emission_prob = {}
    tag_count_model = {}

    try:
        with codecs.open(fin, mode='r', encoding="utf-8") as input_file:
            lines = input_file.readlines()

        # Determine sections by finding header lines
        try:
            emission_start = lines.index("Emission Model\n")
            tag_count_start = lines.index("Tag Counts\n")
        except ValueError:
            print("Error: Model file format is incorrect. Headers not found.")
            sys.exit()

        # Parse Transition Probabilities
        for line in lines[1:emission_start]:
            line = line.strip()
            if ':' in line:
                key, value = line.rsplit(':', 1)
                transition_prob[key] = Decimal(value)

        # Parse Emission Probabilities
        word_set = set()
        tag_set = set()
        for line in lines[emission_start + 1:tag_count_start]:
            line = line.strip()
            if ':' in line:
                key, value = line.rsplit(':', 1)
                emission_prob[key] = Decimal(value)
                word, tag = key.rsplit('/', 1)
                word_set.add(word)
                tag_set.add(tag)

        # Parse Tag Counts
        for line in lines[tag_count_start + 1:]:
            line = line.strip()
            if ':' in line:
                key, value = line.rsplit(':', 1)
                tag_count_model[key] = int(value)

        tag_list = list(tag_set)
        return tag_list, transition_prob, emission_prob, tag_count_model, word_set

    except IOError:
        print(f"File not found: {fin}")
        sys.exit()


# --- MODIFIED: The Viterbi algorithm now uses the lexicon and suffix model ---
def viterbi_algorithm(sentence, tag_list, transition_prob, emission_prob, tag_count, word_set, lexicon, suffix_model):
    sentence = sentence.strip()
    word_list = sentence.split(" ")
    viterbi_matrix = [{}]
    backpointer = [{}]

    # --- Step 1: Initialization ---
    for tag in tag_list:
        tp = transition_prob.get("start~tag~" + tag, LAMBDA)

        word_lower = word_list[0].lower()
        em_key = word_lower + "/" + tag

        if em_key in emission_prob:
            em = emission_prob[em_key]
        else: # Handle unknown word
            # Try suffix model first
            suffix = word_lower[-3:]
            if suffix in suffix_model and tag in suffix_model[suffix]:
                em = suffix_model[suffix][tag]
            else: # Fallback to Lidstone smoothing
                em = LAMBDA / (tag_count.get(tag, 0) + (LAMBDA * len(word_set)))

        viterbi_matrix[0][tag] = tp * em
        backpointer[0][tag] = "start"

    # --- Step 2: Recursion ---
    for t in range(1, len(word_list)):
        viterbi_matrix.append({})
        backpointer.append({})
        word_lower = word_list[t].lower()

        # --- NEW: Use lexicon to prune the tags we consider ---
        possible_tags = lexicon.get(word_lower, tag_list)

        for tag in possible_tags:
            max_prob = Decimal(0)
            best_prev_tag = None

            em_key = word_lower + "/" + tag
            if em_key in emission_prob:
                em = emission_prob[em_key]
            else: # Handle unknown word
                suffix = word_lower[-3:]
                if suffix in suffix_model and tag in suffix_model[suffix]:
                    em = suffix_model[suffix][tag]
                else:
                    em = LAMBDA / (tag_count.get(tag, 0) + (LAMBDA * len(word_set)))

            # Find the best path to the current tag
            for prev_tag in viterbi_matrix[t-1]:
                tp = transition_prob.get(prev_tag + "~tag~" + tag, LAMBDA)
                prob = viterbi_matrix[t-1][prev_tag] * tp
                if prob > max_prob:
                    max_prob = prob
                    best_prev_tag = prev_tag

            viterbi_matrix[t][tag] = max_prob * em
            backpointer[t][tag] = best_prev_tag

    # --- Step 3: Termination ---
    best_last_tag = max(viterbi_matrix[-1], key=viterbi_matrix[-1].get)
    tag_sequence = [best_last_tag]

    # --- Step 4: Backtracking ---
    for t in range(len(word_list) - 1, 0, -1):
        prev_tag = backpointer[t][tag_sequence[0]]
        tag_sequence.insert(0, prev_tag)

    return " ".join(tag_sequence)


# --- MODIFIED: Main Test Execution Block ---
# Note: The new models (lexicon, suffix_model) are loaded from the previous cell's execution state.
tag_list, transition_model, emission_model, tag_count_model, word_set = parse_models()

input_file = codecs.open("test_input.txt", mode='r', encoding="utf-8")
fout = codecs.open("hmmoutput.txt", mode='w', encoding="utf-8")

for sentence in input_file.readlines():
    if sentence.strip() == "":
        continue
    # --- MODIFIED: Pass the new models to the Viterbi function ---
    path = viterbi_algorithm(sentence, tag_list, transition_model, emission_model, tag_count_model, word_set, lexicon, suffix_model)

    words = sentence.strip().split(" ")
    tags = path.strip().split(" ")

    output_line = []
    for j in range(len(words)):
        output_line.append(f"{words[j]}/{tags[j]}")

    fout.write(" ".join(output_line) + "\n")

input_file.close()
fout.close()

print("Tagging complete. hmmoutput.txt generated.")

Tagging complete. hmmoutput.txt generated.


In [29]:
import codecs

def read_tagged_file(filename):
    with codecs.open(filename, mode='r', encoding='utf-8') as f:
        lines = f.readlines()
        tag_data = []
        for line in lines:
            tokens = line.strip().split()
            for token in tokens:
                if '/' in token:
                    word, tag = token.rsplit('/', 1)
                    tag_data.append(tag)
        return tag_data

# Ground truth (gold standard)
true_tags = read_tagged_file("test_gold.txt")

# Your model's output
predicted_tags = read_tagged_file("hmmoutput.txt")

# Ensure both lengths match
if len(true_tags) != len(predicted_tags):
    print("Mismatch in token counts between gold and predicted!")
else:
    correct = sum(1 for t1, t2 in zip(true_tags, predicted_tags) if t1 == t2)
    total = len(true_tags)
    accuracy = (correct / total) * 100
    print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 81.57%


In [24]:

import codecs

def evaluate_accuracy(pred_file="hmmoutput.txt", gold_file="test_gold.txt"):
    with codecs.open(pred_file, "r", encoding="utf-8") as pf, codecs.open(gold_file, "r", encoding="utf-8") as gf:
        pred_lines = pf.readlines()
        gold_lines = gf.readlines()

    correct = total = 0
    for pred, gold in zip(pred_lines, gold_lines):
        pred_tokens = pred.strip().split()
        gold_tokens = gold.strip().split()
        for p, g in zip(pred_tokens, gold_tokens):
            if p == g:
                correct += 1
            total += 1

    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy: {accuracy*100:.2f}%")

# Run accuracy check
evaluate_accuracy()


Accuracy: 18.85%
